In [1]:
import json
import os
import smtplib
import tkinter as tk
import requests
import random
from tkinter import messagebox
import time
import datetime
import smtplib
from email.message import EmailMessage

In [2]:
res = requests.get('https://opentdb.com/api.php?amount=50&category=9&difficulty=easy&type=multiple')
data = json.loads(res.text)
results = data['results']
results = random.sample(results, 20)
#results

In [3]:
name = ""
email = ""
i = 0
start_time = 0
finish_time = 0
choice = ''
question = ''
answers = []
letters = 'ABCD'
ans_index = ''
mark = 0
f = open('Quiz.txt', 'w', encoding ='utf-8')

In [4]:
def start():
    if (txt_name.get() != '' and txt_email.get() != ''):
        global name, email
        name = txt_name.get()
        email = txt_email.get()
        lbl_name.forget()
        txt_name.forget()
        lbl_email.forget()
        txt_email.forget()
        btn_start.forget()
        global start_time, f
        f.write(f'NAME: {name}\n')
        start_time = time.time()
        next_()
    else:
        messagebox.showerror('ERROR!!!', 'Name or Email address field cannot be empty.')

In [5]:
window = tk.Tk()
window.title('Quiz Application')
window.geometry('800x350')
lbl_name = tk.Label(text = 'Name:', font = ('Georgia', 15))
lbl_email = tk.Label(text = 'Email:', font = ('Georgia', 15))
txt_name = tk.Entry(width = 50)
txt_name.focus()
txt_email = tk.Entry(width = 50)
btn_start = tk.Button(text= 'Start Quiz', command = start, font = ('Georgia', 15))
lbl_name.pack(anchor ='n')
txt_name.pack(anchor ='n')
lbl_email.pack(anchor ='n')
txt_email.pack(anchor ='n')
btn_start.pack(anchor ='n', pady=10)

In [6]:
selected = tk.StringVar(value = ' ')

In [7]:
def radio_click():
    global selected, choice
    choice = selected.get()

In [8]:
lbl_num = tk.Label(text = "", font = ('Georgia', 15))
lbl_que = tk.Label(text = '', font = ('Georgia', 15))
rdo_A = tk.Radiobutton(text = '', variable = selected, command = radio_click, font = ('Georgia', 15))
rdo_B = tk.Radiobutton(text = '', variable = selected, command = radio_click, font = ('Georgia', 15))
rdo_C = tk.Radiobutton(text = '', variable = selected, command = radio_click, font = ('Georgia', 15))
rdo_D = tk.Radiobutton(text = '', variable = selected, command = radio_click, font = ('Georgia', 15))
btn_next = tk.Button(text = 'Next', font = ('Georgia', 15))

In [9]:
def send_mail(attachment):
    username = os.environ.get('User_name')
    user_password = os.environ.get('User_password')
    msg = EmailMessage()
    msg['TO'] = email
    msg['From'] = username
    msg['Subject'] = 'Quiz Result'
    msg.set_content (f'''Dear {name},
    Thank you for completing the quiz exercise. 
    You scored {mark*5}%.
    Please, find attached a copy of your results. 
    Thank you.
    ''')
    attachment = attachment.encode('utf-8')
    msg.add_attachment(attachment, maintype = 'application', subtype ='octet-stream', filename='Quiz.txt')
    
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        #try:
            smtp.login(username, user_password)
            smtp.send_message(msg)
        #except gaierror:
            #print('Please, connect to the internet.')

In [10]:
def next_():
    global i, selected, choice, question, answers, ans_index, letters,f
    global lbl_num, lbl_que, rdo_A,rdo_B,rdo_C,rdo_D, btn_next, mark
    
    if(i > 0):
        f.write(f'{i}. {question}\n')
        
        if (choice !=''):
            f.write(f"Your answer: {choice}. {answers[letters.index(choice)]}\n")
        else:
            f.write(f"Your answer: {choice}\n")
                    
        if choice == ans_index:
            mark+=1
        else:
            f.write(f"Correct answer: {ans_index}. {answers[letters.index(ans_index)]}\n")
        choice = ''
        
    if ( i == 20):
        global finish_time
        finish_time = time.time()
        #print(finish_time - start_time)
        t = finish_time - start_time
        h = int(t//3600)
        m = int((t%3600)//60)
        s = int(round((t%3600)%60, 0))
        duration = f"{h}hours, {m}mins and {s}secs"
        message = f"Thank you for completing the assessment. You finished in {duration}"
        lbl_num.configure(text=message)
        #print(mark)
        lbl_que.configure(text=f"Score: {mark * 5}%")
        rdo_A.forget()
        rdo_B.forget()
        rdo_C.forget()
        rdo_D.forget()
        btn_next.forget()
        f.close()
        with open('Quiz.txt') as file_obj:
            send_mail(file_obj.read())
        return
    
    question = results[i]['question']
    correct_ans = [results[i]['correct_answer']]
    incorrect_ans = results[i]['incorrect_answers']
    answers = incorrect_ans+correct_ans
    random.shuffle(answers)
    ans_index = letters[answers.index(correct_ans[0])]    
    
    selected.set(None)
    
    lbl_num.configure(text=f"Question {i+1}:")
    lbl_que.configure(text=question)
    rdo_A.configure(text=answers[0], value = letters[0])
    rdo_B.configure(text=answers[1], value = letters[1])
    rdo_C.configure(text=answers[2], value = letters[2])
    rdo_D.configure(text=answers[3], value = letters[3])
    btn_next.configure(command = next_)
    
        
    if(i==0):
        lbl_num.pack(anchor = 'nw')
        lbl_que.pack(anchor = 'n')
        rdo_A.pack()
        rdo_B.pack()
        rdo_C.pack()
        rdo_D.pack()
        btn_next.pack()
    
    i+=1

In [11]:
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "<ipython-input-9-d4574dd17180>", line 19, in send_mail
    smtp.login(username, user_password)
  File "C:\Users\OLUBAYODE\anaconda3\lib\smtplib.py", line 723, in login
    (code, resp) = self.auth(
  File "C:\Users\OLUBAYODE\anaconda3\lib\smtplib.py", line 642, in auth
    authobject(challenge).encode('ascii'), eol='')
AttributeError: 'NoneType' object has no attribute 'encode'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\OLUBAYODE\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-10-5e8555673574>", line 39, in next_
    send_mail(file_obj.read())
  File "<ipython-input-9-d4574dd17180>", line 20, in send_mail
    smtp.send_message(msg)
  File "C:\Users\OLUBAYODE\anaconda3\lib\smtplib.py", line 282, in __exit__
    raise SMTPResponseException(code, message)
smtplib.SMTPResponseExc